In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from ipywidgets import (GridspecLayout, GridBox , VBox, HBox, HTML, Layout, Text, 
                        Button, Output, Checkbox, Label)
from HUGS.Processing import search
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Axis, Lines, Figure
from random import randint    
from numpy import random as np_random
import numpy as np

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Start with uploading and processing of the data

In [ ]:
# These layouts are used throughout the notebook for consistent sizing of objects
table_style = {'description_width': 'initial'}
table_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}
date_layout = {'width':'275px', 'min_width':'200px', 'height':'28px', 'min_height':'28px'}
checkbox_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}
statusbar_layout = {'width':'250px', 'min_width':'250px', 'height':'28px', 'min_height':'28px'}

In [ ]:
login_text = HTML(value="<b>Please enter your username to be taken to a login page</b>")
username_text = Text(value=None, placeholder="user", description="Username: ")
login_button = Button(description="Login", button_style="success")
status_text = HTML(value="")
login_link_box = Output()

user = None
def login(a):
    global user
    base_url = "https://hugs.acquire-aaai.com/t"
    if username_text.value:
        status_text.value = f"<font color='blue'>Please wait for login link...</font>"
        user = User(username=username_text.value, identity_url=F"{base_url}/identity")
        with login_link_box:
            login_link_box.clear_output()
            response = user.request_login()
                
        if user.wait_for_login():
            status_text.value = f"<font color='green'>Success</font>"
        else:
            status_text.value = f"<font color='red'>Failure</font>"
            
    hugs = Service(service_url="%s/hugs" % base_url)
    _ = hugs.call_function(function="clear_datasources", args={})
            
        
login_button.on_click(login)
    
# user_box = VBox(children=[login_text, username_text, login_button, status_text, login_link_box])
login_children = [login_text, username_text, login_button, status_text, login_link_box]

Check we're logged in

In [ ]:
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

### Upload files for processing


Clear the Datasources from CRDS and GC objects (currently Datasource UUIDs are just being randomly generated and we get multiple Datasources holding the same data)

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"



In [ ]:
def upload_bsd(arg):
    from HUGS.Client import Process
    
    filename = "bsd.picarro.1minute.248m.dat"
    filepath = get_CRDS_path(filename)
    processing = Process(service_url=base_url)
    with output_box:
        print(F"Uploading {filename}")
    result = processing.process_files(user=user, files=filepath, data_type="CRDS")
    
    with output_box:
        print(result)
    
def upload_hfd(arg):
    from HUGS.Client import Process
    
    filename = "hfd.picarro.1minute.100m_min.dat"
    filepath = get_CRDS_path(filename)
    processing = Process(service_url=base_url)
    with output_box:
        print(F"Uploading {filename}")
    result = processing.process_files(user=user, files=filepath, data_type="CRDS")
    
    with output_box:
        print(result)

bsd_button = Button(description="Upload BSD", button_style="info")
hfd_button = Button(description="Upload HFD", button_style="info")
output_box = Output()

bsd_button.on_click(upload_bsd)
hfd_button.on_click(upload_hfd)

instructions = HTML(value=f"Please click the buttons below to upload data to the object store.")

# load_box = VBox(children=[instructions, bsd_button, hfd_button, output_box])
upload_children = [instructions, bsd_button, hfd_button, output_box]
# upload_box

In [ ]:
from HUGS.Processing import search

def parse_results(results):
    """ Split the keys into a list of each key and the date that the data covers
        
        Args:
            results (dict): Dictionary of search results
        Returns:
            list (tuple): List of date, data key list pairs
    """
    date_keys = {}
    for key in results.keys():
        keys = sorted(results[key])
        start_key = keys[0]
        end_key = keys[-1]
        # Get the first and last dates from the keys in the search results
        start_date = start_key.split("/")[-1].split("_")[0]
        end_date = end_key.split("/")[-1].split("_")[-1]
        
        dates_covered = start_date + "_" + end_date
        
        date_keys[key] = {"dates": dates_covered, "keys": keys}
        
    return date_keys

date_keys = None
def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    from datetime import datetime
    from Acquire.ObjectStore import datetime_to_string
    from HUGS.Client import Search
    start = datetime(1970, 1,1) # datetime.combine(start_picker.value, datetime.min.time())
    end = datetime.now() # datetime.combine(end_picker.value, datetime.min.time())
    
    split_search_terms = search_terms.value.replace(" ", "").split(",")
    split_locations = locations.value.replace(" ", "").split(",")
    
    global search_results
    global date_keys
    
    search = Search(service_url=base_url)
    search_results = search.search(search_terms=split_search_terms, locations=split_locations, data_type=data_type.value, start_datetime=start, end_datetime=end)
    
    if search_results:
        date_keys = parse_results(search_results)
        status_box.value = f"<font color='green'>Success</font>"
        # Now we have search results we can select the ones we want to download
        create_download_box()
    else:
        status_box.value = f"<font color='red'>No results</font>"   

search_results = None

search_terms = widgets.Text(value="", placeholder = "Search", description = "Search terms:", disabled = False)
locations = widgets.Text(value="", placeholder = "BSD, HFD", description = "Locations:", disabled = False)
data_type = widgets.Dropdown(options=["CRDS", "GC"], value="CRDS", description="Data type", disabled=False)

# search_layout = widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success")
# layout=widgets.Layout(flex='1 1 0%', width='25%')



start_picker = widgets.DatePicker(description='Start date',disabled=False)
end_picker = widgets.DatePicker(description='End date',disabled=False)

status_box = widgets.HTML(value="")

search_children = [search_terms, locations, start_picker, end_picker, data_type,
                                 search_button, status_box]
# search_box.layout = search_layout


     
    
search_button.on_click(call_search)



Now take the result dictionary, parse the dictionary into a blocks that can be selected by the user

Get this to only be drawn after the 

In [ ]:
def create_download_box():
    """ Creates the plotting box that holds the plotting buttons and windows
    
    """
    
    header_label_site = HTML(value=f"<b>Site</b>", layout=table_layout)
    header_label_gas = HTML(value=f"<b>Gas</b>", layout=table_layout)
    header_label_dates = HTML(value=f"<b>Dates</b>", layout=date_layout)
    header_label_select = HTML(value=f"<b>Select</b>", layout=checkbox_layout)

    checkbox_objects = []
    search_keys = []

    site_labels = []
    date_labels = []
    gas_labels = []
    for key in date_keys:
        # Create the checkboxes
        checkbox = Checkbox(value=False)
        checkbox_objects.append(checkbox)
        search_keys.append(key)

        dates = date_keys[key]["dates"].replace("_", " to ").replace("T", " ")
        date_label = Label(value=dates, layout=date_layout)

        split_key = key.split("_")
        site_name = split_key[0].upper()
        gas_name = split_key[1].upper()

        gas_label = Label(value=gas_name, layout=table_layout)
        site_label = Label(value=site_name, layout=table_layout)

        date_labels.append(date_label)
        site_labels.append(site_label)
        gas_labels.append(gas_label)


    arg_dict = {search_keys[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

    header_box = HBox(children=[header_label_site, header_label_gas, header_label_dates, header_label_select])

    site_vbox = VBox(children=site_labels)
    gas_vbox = VBox(children=gas_labels)
    dates_vbox = VBox(children=date_labels)
    checkbox_vbox = VBox(children=checkbox_objects)

    dynamic_box = HBox(children=[site_vbox, gas_vbox, dates_vbox, checkbox_vbox])

    download_button = Button(description="Download", button_style="success", layout=table_layout)
    download_button_box = HBox(children=[download_button])

    status_bar = HTML(value="Status: Waiting...", layout=statusbar_layout)

#     complete = VBox(children=[header_box, dynamic_box, download_box, status_bar])
    
    download_box.children = [header_box, dynamic_box, download_button_box, status_bar]

    selected_data = []
    def select_data(**kwargs):
        selected_data.clear()

        for key in kwargs:
            if kwargs[key] is True:
                selected_data.append(key)

    def update_statusbar(text):
        status_bar.value = F"Status: {text}"

    data = None
    def download_data(a):
        """ Download the data in the selected keys from the object store

            Returns:
                Pandas.Dataframe of selected data
        """
        from HUGS.Client import Retrieve
        from pandas import read_json as pd_read_json

        update_statusbar("Downloading...")

        download_keys = {key: date_keys[key]["keys"] for key in selected_data}

        retrieve = Retrieve(service_url=base_url)

        global data
        data = retrieve.retrieve(keys=download_keys)

        # Conver the JSON into Dataframes
        for key in data:
            data[key] = pd_read_json(data[key])

        # Update the status bar
        if data:
            update_statusbar("Download complete")
            # Create the plotting box
            create_plotting_box()
        else:
            update_statusbar("No data downloaded")

    download_button.on_click(download_data)
    out = widgets.interactive_output(select_data, arg_dict)

#     display(complete, out)

Plot the data we've selected

Select the data from the downloaded data using the checkboxes and then click plot to update the figure

In [ ]:
def create_plotting_box():
    """ Create the window for plotting the downloaded data
            
    """
    # Create some checkboxes
    plot_checkboxes = []
    plot_keys = [] 

    for key in data:
        # Create a more readable description
        desc = " ".join(key.split("_")).upper()
        plot_keys.append(key)
        plot_checkboxes.append(Checkbox(description=desc, value=False))

    select_instruction = HTML(value="<b>Select data: </b>", layout=table_layout)
    plot_button = Button(description="Plot", button_style="success", layout=table_layout)

    select_box = HBox(children=[select_instruction])
    checkbox_box = VBox(children=plot_checkboxes)
    horiz_select = HBox(children=[select_box, checkbox_box])
    plot_box = HBox(children=[plot_button])
    ui_box = VBox(children=[horiz_select, plot_box])

    arg_dict = {plot_keys[i]: checkbox for i, checkbox in enumerate(plot_checkboxes)}

    selected_data = []
    def select_data(**kwargs):
        selected_data.clear()

        for key in kwargs:
            if kwargs[key] is True:
                selected_data.append(key)

        print(selected_data)

    # Link the checkbox selection with the selected dataframes to plot
    out = widgets.interactive_output(select_data, arg_dict)

    output = widgets.Output()

    def plot_data(arg):
        """ Each key in the data dict is a dataframe

        """
        # Here take the keys in the selected data list and use them to
        # access the Dataframes to plot
        # Use the same axes. Can have a button to create new plots etc in the future

        # TODO - change this to take the data directly from the dict?
        plot_data = [data[x] for x in selected_data]

        # For now just plot the first column in the data

         # Setup the axes    

        x_scale = DateScale()
        y_scale = LinearScale()
        scales = {"x": x_scale, "y": y_scale}

        r = lambda: randint(0,255)
    #     lines = [Lines(x=d.index.values, y=d.iloc[:,0], scales=scales, colors=['#%02X%02X%02X' % (r(),r(),r())]) for d in plot_data]
    #     lines.x = plot_data[0].index.values
    #     fig.marks = [Lines(x=d.index.values, y=d.iloc[:,0], scales=scales, colors=['#%02X%02X%02X' % (r(),r(),r())]) for d in plot_data]
#         print(type(plot_data[0].index.values))

        lines.x = [d.index.values.tolist() for d in plot_data]    
        lines.y = [d.iloc[:,0] for d in plot_data]

    #     print(plot_data[0].index.values.tolist())

    #     print(type(plot_data[0].first_valid_index()))

    #     print(lines.x)


    #     lines.x = np.arange(100)
    #     lines.y = np.cumsum(np.random.randn(2, 100), axis=1)
    #     lines.x = [Lines(x=d.index.values, y=d.iloc[:,0], scales=scales, colors=['#%02X%02X%02X' % (r(),r(),r())]) for d in plot_data]


    x_scale = DateScale()
    y_scale = LinearScale()
    scales = {"x": x_scale, "y": y_scale}

    ax = Axis(label="Date", scale=x_scale)
    # TODO - this could be updated depending on what's being plot
    ay = Axis(label="Count", scale=y_scale, orientation="vertical")

    lines = Lines(x=np.arange(100), y=np.cumsum(np.random.randn(2, 100), axis=1), scales=scales)
    figure = Figure(marks=[lines], axes=[ax, ay], animation_duration=1000)

    plot_button.on_click(plot_data)

    # plot_button.on_click(lambda btn: plot_data())

#     figure = VBox([fig])
        
#     display(ui_box, out, figure)

    plotting_box.children = [ui_box, out, figure]
    

In [ ]:
# Example layout

# layout={'border': '1px solid black'}

# layout=Layout(width="auto", grid_area="search")
# Setup the GridBox layout
login_box = VBox(children=login_children)
upload_box = VBox(children=upload_children)
# login_upload_box = VBox(children=[login_box, upload_box])
search_box = VBox(children=search_children, layout=Layout(width="auto", grid_area="search"))
download_box = VBox(children=[], layout=Layout(width="auto", grid_area="download"))
plotting_box = VBox(children=[], layout=Layout(width="auto", grid_area="plotting"))

# Now create the Gridbox layout

grid_layout = Layout(width="90%", grid_template_rows="auto auto", grid_template_columns="50% 50%", 
                    grid_template_areas = """
                    login login search search
                    download download plotting plotting
                    """)

grid = GridspecLayout(5, 4)

grid[:1,:] = login_box
grid[1:3, :2] = upload_box
grid[1:3, 2:] = search_box
grid[3:, :3] = download_box
grid[3:, 3:] = plotting_box


# grid = GridBox(children = [login_upload_box, search_box, download_box, plotting_box], layout=grid_layout)
# grid = GridBox(children = [login_upload_box], layout=grid_layout)

display(grid)

In [ ]:
# fig.animation_duration

In [ ]:
    # from bqplot.scales import ColorScale
    # print(ColorScale.colors)

In [ ]:
# plot_checkboxes

Now we can plot some of the data

In [ ]:
# import pandas as pd
# # co2_data = pd.read_json(data["results"]["bsd_co2"])
# co_data = pd.read_json(data["results"]["bsd_ch4"])


In [ ]:
# co_data.head(10)

In [ ]:
# %matplotlib notebook
# import matplotlib.pyplot as plt
# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

# fig = plt.figure()
# # ax = fig.add_subplot(211)
# # ax.plot(co2_data.index.values, co2_data["co2 count"], color = "#4e79a7" )
# ax = fig.add_subplot(111)
# ax.plot(co_data.index.values, co_data["ch4 count"], color = "#59a14f")

# plt.show()

In [ ]:
# Checkboxes for selecting which data we want to plot
# Select the gas, it goes and plots the 
# Simple line plot

In [ ]:
# from ipyleaflet import (
#     Map,
#     Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
#     Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
#     SplitMapControl, WidgetControl,
#     basemaps, basemap_to_tiles
# )

# from ipywidgets import HTML

In [ ]:
# center = [54.2361, -4.548]
# zoom = 5
# m = Map(center=center, zoom=zoom)
# mark_bsd = Marker(location=(54.942544, -1.369204))
# mark_bsd.popup = HTML(value="Bilsdale")

# mark_mhd = Marker(location=(53.20,-9.54))
# mark_mhd.popup = HTML(value="Macehead")

# mark_tac = Marker(location=(52.511, 1.155003))
# mark_tac.popup = HTML(value="Tacolneston")

# m += mark_bsd
# m += mark_mhd
# m += mark_tac

# display(m)

In [ ]:
# m